# Joins

In the previous section, we talked about the motivation of Fugue and showed a simple example of Fugue code. Here we provide more details around the Fugue syntax that will be needed in actual applications. We will also talk about some of the underlying concepts that were shown in the previous section, but not really explained.

## The Directed Acyclic Graph (DAG)

In the last section, we had a code block that used `with FugueWorkflow() as dag:`. We'll make another simple example below. Here we have a simple `transformer` that takes in two columns to create a third column called `col3`. This is then used in the `FugueWorkflow`.